In [ ]:
import hexaly.optimizer
import util

In [ ]:
import marimo as mo
import nbformat

# Hexaly を使ったヒューリスティック

Hexaly の外部関数最適化機能を用いた定式化を紹介する.
Weighted Majority Merge アルゴリズムにおいて次の文字を選択する基準は各文字列に対する残長の和であった.
その残長の和の部分を Hexaly の決定変数で置き換える.

**決定変数**

- $w_{ij} \in \mathbb{N}$: 文字列 $s_i$ の $j$ 文字目の重み. $(i \in \{ 1, \dots, n \}, \ j \in \{ 1, \dots, |s_i| \})$
    - $w_i = \{ w_{i,1} \dots, w_{i,|s_i|} \}$ とおく.


**目的関数**

下記のアルゴリズムに従って構築した共通超配列の長さを目的関数とする.

- 解 $\mathrm{sol}$ を空文字列で初期化する.
- 各文字 $c$ に対して重み $\sum_{i=1, \ s_i[0] = c}^n w_{i,1}$ を計算し, 重みが最大である $c$ を求める.
- $\mathrm{sol}$ の後ろに $c$ を追加する.
- 各文字列 $s_i \ (i \in \{ 1, \dots, n \})$ に対し, 先頭の文字が $c$ である場合は
    - $s_i$ の先頭の文字を削除する.
    - $w_i$ の先頭の重みを削除し, インデックスを前に詰める.
- $s_1, \dots, s_n$ 全てが空文字列になれば終了. $\mathrm{sol}$ が解.

In [ ]:
class Model:
    def __init__(self, instance):
        chars = sorted(list(set("".join(instance))))

        hxoptimizer = hexaly.optimizer.HexalyOptimizer()
        hxmodel = hxoptimizer.model

        priorities1d = [
            hxmodel.int(1, len(chars)) for s in instance for _ in s
        ]

        func = hxmodel.create_int_external_function(self.objective)
        func.external_context.lower_bound = 0
        func.external_context.upper_bound = sum(len(s) for s in instance)

        self.instance = instance
        self.chars = chars
        self.hxoptimizer = hxoptimizer
        self.hxmodel = hxmodel
        self.priorities1d = priorities1d

        # これらが実行される時点で self.* が必要になるため初期化の最後に移動
        hxmodel.minimize(func(*priorities1d))
        hxmodel.close()

    def solve(self, time_limit: int | None = 60, log: bool = False) -> "Model":
        if time_limit is not None:
            self.hxoptimizer.param.time_limit = time_limit
        self.hxoptimizer.param.verbosity = 1 if log else 0
        self.hxoptimizer.solve()
        return self

    def to_solution(self) -> str | None:
        status = self.hxoptimizer.solution.status
        if status not in {
            hexaly.optimizer.HxSolutionStatus.OPTIMAL,
            hexaly.optimizer.HxSolutionStatus.FEASIBLE,
        }:
            return None

        priorities1d_value = [priority.value for priority in self.priorities1d]
        priorities2d_value = self.priorities_1d_to_2d(priorities1d_value)
        return self.make_csp(priorities2d_value)

    def make_csp(self, priorities2d: list[list[int]]) -> str:
        indices = [0] * len(self.instance)
        solution = ""

        # while not all(idx == len(s) for idx, s in zip(indices, self.instance)):
        for _ in range(
            len(self.instance) * max(len(s) for s in self.instance)
        ):
            if all(idx == len(s) for idx, s in zip(indices, self.instance)):
                break

            counts = [
                sum(
                    priorities2d[sidx][idx]
                    for sidx, (idx, s) in enumerate(
                        zip(indices, self.instance)
                    )
                    if idx < len(s) and s[idx] == c
                )
                for c in self.chars
            ]
            next_char = self.chars[counts.index(max(counts))]

            solution += next_char
            for sidx, s in enumerate(self.instance):
                idx = indices[sidx]
                if idx < len(s) and s[idx] == next_char:
                    indices[sidx] += 1

        return solution

    def priorities_1d_to_2d(self, priorities1d: list[int]) -> list[list[int]]:
        counter = 0
        priorities2d = []
        for s in self.instance:
            priorities2d.append([])
            for _ in s:
                priorities2d[-1].append(priorities1d[counter])
                counter += 1

        return priorities2d

    def objective(self, priorities1d: list[int]):
        priorities2d = self.priorities_1d_to_2d(priorities1d)
        solution = self.make_csp(priorities2d)
        return len(solution)

In [ ]:
def solve(
    instance: list[str], time_limit: int | None = 60, log: bool = False
) -> str | None:
    return Model(instance).solve(time_limit, log).to_solution()

In [ ]:
instance_01 = util.parse("uniform_q26n004k015-025.txt")
solution_01 = solve(instance_01, log=True)


Model:  expressions = 88, decisions = 84, constraints = 0, objectives = 1
Param:  time limit = 60 sec, no iteration limit

[objective direction ]:     minimize

[  0 sec,       0 itr]:           74
[ optimality gap     ]:      100.00%


[  1 sec,     142 itr]:           70


[  2 sec,     854 itr]:           64


[  3 sec,    1585 itr]:           63


[  4 sec,    2343 itr]:           62


...

[ 56 sec,   42220 itr]:           62


[ 57 sec,   42989 itr]:           62


[ 58 sec,   43758 itr]:           62


[ 59 sec,   44527 itr]:           62


[ 60 sec,   45293 itr]:           62
[ optimality gap     ]:      100.00%
[ 60 sec,   45293 itr]:           62
[ optimality gap     ]:      100.00%

45293 iterations performed in 60 seconds

Feasible solution: 
  obj    =           62
  gap    =      100.00%
  bounds =            0


In [ ]:
_instance = instance_01
_solution = solution_01

util.show(_instance)
if _solution is not None:
    util.show(_instance, _solution)
    print(f"solution is feasible: {util.is_feasible(_instance, _solution)}")
else:
    print("--- Solution not found ---")

--- Condition (with 25 chars) ---
str1: tkgnkuhmpxnhtqgxzvxis
str2: iojiqfolnbxxcvsuqpvissbxf
str3: ulcinycosovozpplp
str4: igevazgbrddbcsvrvnngf

--- Solution (of length 62) ---
 Sol: tkulcigneycojiqfsovoazpkuhmplxnhtqgbxzrddbxcvsxuqpvissbxrvnngf
str1: tk----gn---------------kuhmp-xnhtqg-xz------v-x----is---------
str2: -----i-----ojiqf-o----------l-n----bx-----xcvs-uqpvissbx-----f
str3: --ulci-n-yco----sovo-zp----pl--------------------p------------
str4: -----ig-e---------v-az------------gb--rddb-c-s----v-----rvnngf

solution is feasible: True


In [ ]:
instance_02 = util.parse("uniform_q26n008k015-025.txt")
solution_02 = solve(instance_02, log=True)


Model:  expressions = 179, decisions = 175, constraints = 0, objectives = 1
Param:  time limit = 60 sec, no iteration limit

[objective direction ]:     minimize

[  0 sec,       0 itr]:          148
[ optimality gap     ]:      100.00%


[  1 sec,      48 itr]:          133


[  2 sec,     302 itr]:          115


[  3 sec,     620 itr]:          113


[  4 sec,     924 itr]:          111


...

[ 56 sec,   17980 itr]:          102


[ 57 sec,   18309 itr]:          102


[ 58 sec,   18633 itr]:          102


[ 59 sec,   18965 itr]:          102


[ 60 sec,   19304 itr]:          102
[ optimality gap     ]:      100.00%
[ 60 sec,   19304 itr]:          102
[ optimality gap     ]:      100.00%

19304 iterations performed in 60 seconds

Feasible solution: 
  obj    =          102
  gap    =      100.00%
  bounds =            0


In [ ]:
_instance = instance_02
_solution = solution_02

util.show(_instance)
if _solution is not None:
    util.show(_instance, _solution)
    print(f"solution is feasible: {util.is_feasible(_instance, _solution)}")
else:
    print("--- Solution not found ---")

--- Condition (with 26 chars) ---
str1: tkgnkuhmpxnhtqgxzvxis
str2: iojiqfolnbxxcvsuqpvissbxf
str3: ulcinycosovozpplp
str4: igevazgbrddbcsvrvnngf
str5: pyplrzxucpmqvgtdfuivcdsbo
str6: pbdevdcvdpfzsmsbroqvbbh
str7: enbczfjtvxerzbrvigple
str8: rxwxqkrdrlctodtmprpxwd

--- Solution (of length 102) ---
 Sol: ipuyplrogenbdcevadzfxwgbjtinyucvxerdpmqfozkrdbrlcsovgtondfkuibhtmpxrvnhntqgxzpcvxwdsubroqpvisslbxfbhep
str1: -------------------------t----------------k---------g--n--ku--h-mpx--nh-tqgxz--vx----------is---------
str2: i------o----------------j-i-----------qfo------l-------n-----b----x--------x--cv---su---qpviss-bxf----
str3: --u--l-------c------------iny-c---------o--------sov--o---------------------zp-----------p----l------p
str4: i-------ge-----va-z---gb----------rd--------db--cs-v---------------rvn-n--g----------------------f----
str5: -p-yplr-----------z-x--------uc-----pmq------------vgt--df-ui-------v---------c---ds-b-o--------------
str6: -p---------bd-ev-d------------cv---dp--f-

In [ ]:
instance_03 = util.parse("uniform_q26n016k015-025.txt")
solution_03 = solve(instance_03, log=True)


Model:  expressions = 327, decisions = 323, constraints = 0, objectives = 1
Param:  time limit = 60 sec, no iteration limit

[objective direction ]:     minimize

[  0 sec,       0 itr]:          198
[ optimality gap     ]:      100.00%


[  1 sec,       7 itr]:          189


[  2 sec,     177 itr]:          172


[  3 sec,     325 itr]:          170


[  4 sec,     478 itr]:          169


...

[ 56 sec,    7824 itr]:          157


[ 57 sec,    7971 itr]:          157


[ 58 sec,    8116 itr]:          156


[ 59 sec,    8247 itr]:          156


[ 60 sec,    8388 itr]:          156
[ optimality gap     ]:      100.00%
[ 60 sec,    8388 itr]:          156
[ optimality gap     ]:      100.00%

8388 iterations performed in 60 seconds

Feasible solution: 
  obj    =          156
  gap    =      100.00%
  bounds =            0


In [ ]:
_instance = instance_03
_solution = solution_03

util.show(_instance)
if _solution is not None:
    util.show(_instance, _solution)
    print(f"solution is feasible: {util.is_feasible(_instance, _solution)}")
else:
    print("--- Solution not found ---")

--- Condition (with 26 chars) ---
str01: tkgnkuhmpxnhtqgxzvxis
str02: iojiqfolnbxxcvsuqpvissbxf
str03: ulcinycosovozpplp
str04: igevazgbrddbcsvrvnngf
str05: pyplrzxucpmqvgtdfuivcdsbo
str06: pbdevdcvdpfzsmsbroqvbbh
str07: enbczfjtvxerzbrvigple
str08: rxwxqkrdrlctodtmprpxwd
str09: kkqafigqjwokkskrblg
str10: lxxpabivbvzkozzvd
str11: krifsavncdqwhzc
str12: qaxudgqvqcewbfgijowwy
str13: rsxqjnfpadiusiqbezhkohmg
str14: iwshvhcomiuvddm
str15: htxxqjzqbctbakn
str16: xusfcfzpeecvwantfmgqzu

--- Solution (of length 156) ---
  Sol: irkepxulcswxqkarifsnydhtxpabcuojivncdqgbevzfakolrzgnkbxqujtpvxerdzqcevdwhbcsvrvwantfigplmqvgctdijowfuqpvokiszpsbxnhamiuvdtqgcdbfxezhknsbkroqvbbhmprlpgxwdyis
str01: -----------------------t---------------------k----gnk---u---------------h---------------m-------------p---------xnh------tqg----x-z---------v---------x---is
str02: i-----------------------------oji----q-----f--ol---n-bx------x-----c-v-----s------------------------uqpv--is--sbx--------------f------------------

--------------------
str05: ----p---------------y----p---------------------lrz----x-u----------c------------------p-mqvg-td----fu-----i------------v----cd--------sb--o-----------------
str06: ----p----------------------b--------d---ev----------------------d--c-vd---------------p------------f--------z-s-----m-----------------sb-roqvbbh------------
str07: ---e---------------n-------bc-------------zf-------------jt-vxer-z-------b---rv-----igpl-----------------------------------------e--------------------------
str08: -r---x----wxqk-r-----d--------------------------r--------------------------------------l----ct---o----------------------dt----------------------mpr-p-xwd---
str09: --k----------k-----------------------q------a--------------------------------------fig---q------j-w-----ok--------------------------k-s-kr---b-----l-g------
str10: -------l---x------------xpab----iv-----b-vz--ko--z---------------z---vd---------------------------------------------------------------------------------

In [ ]:
instance_04 = util.parse("uniform_q05n010k010-010.txt")
solution_04 = solve(instance_04, log=True)


Model:  expressions = 104, decisions = 100, constraints = 0, objectives = 1
Param:  time limit = 60 sec, no iteration limit

[objective direction ]:     minimize

[  0 sec,       0 itr]:           32
[ optimality gap     ]:      100.00%


[  1 sec,    2983 itr]:           29


[  2 sec,    6106 itr]:           29


[  3 sec,    9115 itr]:           29


[  4 sec,   12134 itr]:           29


...

[ 56 sec,  171992 itr]:           27


[ 57 sec,  175118 itr]:           27


[ 58 sec,  178215 itr]:           27


[ 59 sec,  181341 itr]:           27


[ 60 sec,  184459 itr]:           27
[ optimality gap     ]:      100.00%
[ 60 sec,  184459 itr]:           27
[ optimality gap     ]:      100.00%

184459 iterations performed in 60 seconds

Feasible solution: 
  obj    =           27
  gap    =      100.00%
  bounds =            0


In [ ]:
_instance = instance_04
_solution = solution_04

util.show(_instance)
if _solution is not None:
    util.show(_instance, _solution)
    print(f"solution is feasible: {util.is_feasible(_instance, _solution)}")
else:
    print("--- Solution not found ---")

--- Condition (with 5 chars) ---
str01: dcbccdbcce
str02: bddbeeeebd
str03: cacdeecebe
str04: aeddddebdd
str05: acbeecabce
str06: bbabebdcba
str07: bbaeaebada
str08: eeeecbdbee
str09: ccdeedadcd
str10: bdabdbeaad

--- Solution (of length 27) ---
  Sol: bbaedcdbeacdeecbdbeacbdacde
str01: ----dc-b--c---c-db--c---c-e
str02: b---d-dbe---ee----e--bd----
str03: -----c---acdeec---e--b----e
str04: --aed-d----d----d-e--bd--d-
str05: --a--c-be---e-c----a-b--c-e
str06: bba----be------bd---cb-a---
str07: bbae-----a--e--b---a--da---
str08: ---e----e---eecbdbe-------e
str09: -----c----cdee--d--a--d-cd-
str10: b---d----a-----bdbea---a-d-

solution is feasible: True


In [ ]:
instance_05 = util.parse("uniform_q05n050k010-010.txt")
solution_05 = solve(instance_05, log=True)


Model:  expressions = 504, decisions = 500, constraints = 0, objectives = 1
Param:  time limit = 60 sec, no iteration limit

[objective direction ]:     minimize

[  0 sec,       0 itr]:           36
[ optimality gap     ]:      100.00%


[  1 sec,     761 itr]:           35


[  2 sec,    1584 itr]:           35


[  3 sec,    2364 itr]:           34


[  4 sec,    3165 itr]:           34


...

[ 56 sec,   43285 itr]:           34


[ 57 sec,   44050 itr]:           34


[ 58 sec,   44828 itr]:           34


[ 59 sec,   45628 itr]:           34


[ 60 sec,   46358 itr]:           34
[ optimality gap     ]:      100.00%
[ 60 sec,   46358 itr]:           34
[ optimality gap     ]:      100.00%

46358 iterations performed in 60 seconds

Feasible solution: 
  obj    =           34
  gap    =      100.00%
  bounds =            0


In [ ]:
_instance = instance_05
_solution = solution_05

util.show(_instance)
if _solution is not None:
    util.show(_instance, _solution)
    print(f"solution is feasible: {util.is_feasible(_instance, _solution)}")
else:
    print("--- Solution not found ---")

--- Condition (with 5 chars) ---
str01: dcbccdbcce
str02: bddbeeeebd
str03: cacdeecebe
str04: aeddddebdd
str05: acbeecabce
str06: bbabebdcba
str07: bbaeaebada
str08: eeeecbdbee
str09: ccdeedadcd
str10: bdabdbeaad
str11: ededaaaeaa
str12: aaeaabeeac
str13: eaabcaccdb
str14: bdeeadeade
str15: caedadeeed
str16: ebcadbabbe
str17: ddceeabdea
str18: dabcddeaec
str19: aadceedaab
str20: aeecceeeaa
str21: bbdaecaade
str22: dacedaedab
str23: aaeabbbbce
str24: dedbcbcaab
str25: dbdaaebbcb
str26: debedbebac
str27: ceebcdcbde
str28: dbedaadaab
str29: cccdcbebdc
str30: aeeacdbcbd
str31: dacbeacccd
str32: ecebccdbdb
str33: ddbbcedabb
str34: aaeabaaeba
str35: ecbbcaadcd
str36: debccecdbc
str37: daacbaeebc
str38: adabeaacce
str39: daecdbacaa
str40: dacbbdcedc
str41: dedbeebbde
str42: cdadcdcdaa
str43: ceedcbaeed
str44: ceaecaaaca
str45: dcccebbbad
str46: baeeaebbde
str47: dbdebaccdb
str48: ebcbeedaea
str49: aeeebbdbca
str50: dbdabcecbb

--- Solution (of length 34) ---
  Sol: daecbdeabecdabcedaecbdaebcd

----
str08: --e---e--e-----e---cbd--b--e-----e
str09: ---c------cd---e--e--da---d----cd-
str10: ----bd-ab--d-b-e-a----a---d-------
str11: --e--de----da----a----ae----a-a---
str12: -a-----a-e--a----a--b--e---ea--c--
str13: --e----a----abc--a-c-----cd--b----
str14: ----bde--e--a---d-e---a---de------
str15: ---c---a-e-da---d-e----e---e----d-
str16: --e-b-----c-a---d---b-a-b----b---e
str17: d----d----c----e--e---a-b-dea-----
str18: da--b-----cd----d-e---ae-c--------
str19: -a-----a---d--ce--e--da-----ab----
str20: -ae---e---c---ce--e----e----a-a---
str21: ----b---b--da--e---c--a-----a---de
str22: da-c--e----da--eda--b-------------
str23: -a-----a-e--ab------b---b----b-c-e
str24: d-e--d--b-c--bc--a----a-b---------
str25: d---bd-a----a--e----b---bc---b----
str26: d-e-b-e----d-b-e----b-a--c--------
str27: ---c--e--e---bc-d--cbd-e----------
str28: d---b-e----da----a---da-----ab----
str29: ---c------c---c-d--cb--eb-d----c--
str30: -ae---ea--cd-bc-----bd------------
str31: da-cb-ea--c---c----c-d

In [ ]:
instance_06 = util.parse("nucleotide_n010k010.txt")
solution_06 = solve(instance_06, log=True)


Model:  expressions = 104, decisions = 100, constraints = 0, objectives = 1
Param:  time limit = 60 sec, no iteration limit

[objective direction ]:     minimize

[  0 sec,       0 itr]:           27
[ optimality gap     ]:      100.00%


[  1 sec,    2500 itr]:           25


[  2 sec,    6467 itr]:           24


[  3 sec,   10238 itr]:           24


[  4 sec,   14008 itr]:           24


...

[ 56 sec,  206042 itr]:           24


[ 57 sec,  209829 itr]:           24


[ 58 sec,  213505 itr]:           24


[ 59 sec,  217262 itr]:           24


[ 60 sec,  221060 itr]:           24
[ optimality gap     ]:      100.00%
[ 60 sec,  221060 itr]:           24
[ optimality gap     ]:      100.00%

221060 iterations performed in 60 seconds

Feasible solution: 
  obj    =           24
  gap    =      100.00%
  bounds =            0


In [ ]:
_instance = instance_06
_solution = solution_06

util.show(_instance)
if _solution is not None:
    util.show(_instance, _solution)
    print(f"solution is feasible: {util.is_feasible(_instance, _solution)}")
else:
    print("--- Solution not found ---")

--- Condition (with 4 chars) ---
str01: ATGGGATACG
str02: ATACCTTCCC
str03: CACGAATTGA
str04: TAAAATCTGT
str05: AGGTAACAAA
str06: TTCCTAGGTA
str07: TTGTAGATCT
str08: TGGGAAGTTC
str09: TTCCACAACT
str10: TCTAAACGAA

--- Solution (of length 24) ---
  Sol: TCATGACCGTAGATCAGTACGATC
str01: --ATG---G--GAT-A---CG---
str02: --AT-ACC-T---TC----C---C
str03: -CA---C-G-A-AT---T--GA--
str04: T-A--A----A-ATC--T--G-T-
str05: --A-G---GTA-A-CA--A--A--
str06: T--T--CC-TAG----GTA-----
str07: T--TG----TAGATC--T------
str08: T---G---G--GA--AGT----TC
str09: T--T--CC--A---CA--AC--T-
str10: TC-T-A----A-A-C-G-A--A--

solution is feasible: True


In [ ]:
instance_07 = util.parse("nucleotide_n050k050.txt")
solution_07 = solve(instance_07, log=True)


Model:  expressions = 2504, decisions = 2500, constraints = 0, objectives = 1
Param:  time limit = 60 sec, no iteration limit

[objective direction ]:     minimize

[  0 sec,       0 itr]:          150
[ optimality gap     ]:      100.00%


[  1 sec,      99 itr]:          143


[  2 sec,     338 itr]:          141


[  3 sec,     579 itr]:          141


[  4 sec,     771 itr]:          138


...

[ 56 sec,    9067 itr]:          136


[ 57 sec,    9192 itr]:          136


[ 58 sec,    9327 itr]:          136


[ 59 sec,    9461 itr]:          136


[ 60 sec,    9586 itr]:          136
[ optimality gap     ]:      100.00%
[ 60 sec,    9586 itr]:          136
[ optimality gap     ]:      100.00%

9586 iterations performed in 60 seconds

Feasible solution: 
  obj    =          136
  gap    =      100.00%
  bounds =            0


In [ ]:
_instance = instance_07
_solution = solution_07

util.show(_instance)
if _solution is not None:
    util.show(_instance, _solution)
    print(f"solution is feasible: {util.is_feasible(_instance, _solution)}")
else:
    print("--- Solution not found ---")

--- Condition (with 5 chars) ---
str01: TAGTAGTAGACTCCGGAAGTGACAAACCCTGAAAAGAATGGATAAATATA
str02: GGATAAACACTCCCGAAAATAATTTGACTTAAACAACGCGACAGTTCAAG
str03: ATACCTTCCTAGGTAACAAACCAACCAACTTTTGATCTCTTGTAGATCTG
str04: TAAATTATAATCTTATACTAGTAAAAAATAGGGTGTAACCGAAAACGGTC
str05: TTAAAACAGCCTGTGGGTTGCACCCACTCACAGGGCCCACTGGGCGCAAG
str06: ATGACTTCCAATGGATCCCAACCTCAAGCTTCCACCCCAATGGTTTCAGC
str07: AACAAACCAACCAACTTTTGATCTCTTGTAGATCTGTTCTCTAAACGAAC
str08: ATGAAAACGAAAATTATTATCAAGGGTATGGAAGTGGAAGCTGACGAAAT
str09: ACTCGGCTGCATGCTTAGTGCACTCACGCAGTATAATTAATAACTAATTA
str10: TTGTAGATCTGTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTCACTC
str11: GCAGAGCATTTTCTAATATCCACAAAATGAAGGCAATAATTGTACTACTC
str12: ATGAGCCAAGATCCGACGAAGAGCCCCAAGGAGGAGAAGGAGGGACCCCC
str13: TCTCACAGTTCAAGAACCCAAAGTACCCCCCATAGCCCTCTTAAAGCCAC
str14: AGGTTTATACCTTCCTAGGTAACAAACCAACCAACTTTCGATCTCTTGTA
str15: AGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGATCTCTTGTA
str16: TAAAACAACTCAATACAACATAAGAAAATCAACGCAAAAACACTCACAAA
str17: CCGCCCATTTGGGCGGCTCTCGAGCGATAGCT

--GA--A-CT--T--T--A--A--A---AT--CTG-T--GT-G----G--C--T--GTCA--C-TC-------------
str11: --G-C--A-G-A--G--CAT---T--T---TC--T--A--A-T-ATC----CA-C-A-A-A--A--T-G--A---A---G---GCA-A--T-A--A-T---T--G-T--A-C--T-A--C-TC-------------
str12: ATGA--G-C----C--A-A-G-ATC---C---GA-CGA--AG--A--G---C--C--C----CA---AG-----GA---G---G-A--G---A--AG--GA---G---G----G--A--C--C--CC-----C---
str13: -T--CT--C--A-C--A---G--T--T-CA---A--GA--A--C--C----CA---A-AG-T-A-C----C--C---C----C-CATAGC----C---C--TCT--T--A--A---AG-C--C-AC----------
str14: A-G---G---T----T---T--AT-A--C--C--T---TC---C-T--A-G--G-TA-A---CA---A---A-C---CA--AC-CA-A-CT--T---TCGATCT---C--T---T--GT-A---------------
str15: A-G---G---T----T---T--AT-A--C--C--T---TC---C--C-A-G--G-TA-A---CA---A---A-C---CA--AC-CA-A-CT--T---TCGATCT---C--T---T--GT-A---------------
str16: -T-A---A---AAC--A-A--C-TCA---AT--A-C-A--A--CAT--A---AG--A-A-A--A--T---CA---A-C-G--C--A-A----A--A----A-C--A-C--TCA--CA---A---A-----------
str17: ----C---CG---C---C---CAT--T---T-G---G----G-C---G-

In [ ]:
instance_08 = util.parse("protein_n010k010.txt")
solution_08 = solve(instance_08, log=True)


Model:  expressions = 104, decisions = 100, constraints = 0, objectives = 1
Param:  time limit = 60 sec, no iteration limit

[objective direction ]:     minimize

[  0 sec,       0 itr]:           62
[ optimality gap     ]:      100.00%


[  1 sec,     334 itr]:           51


[  2 sec,    1104 itr]:           48


[  3 sec,    1931 itr]:           48


[  4 sec,    2625 itr]:           47


...

[ 56 sec,   45272 itr]:           44


[ 57 sec,   46164 itr]:           44


[ 58 sec,   47019 itr]:           44


[ 59 sec,   47876 itr]:           44


[ 60 sec,   48670 itr]:           44
[ optimality gap     ]:      100.00%
[ 60 sec,   48670 itr]:           44
[ optimality gap     ]:      100.00%

48670 iterations performed in 60 seconds

Feasible solution: 
  obj    =           44
  gap    =      100.00%
  bounds =            0


In [ ]:
_instance = instance_08
_solution = solution_08

util.show(_instance)
if _solution is not None:
    util.show(_instance, _solution)
    print(f"solution is feasible: {util.is_feasible(_instance, _solution)}")
else:
    print("--- Solution not found ---")

--- Condition (with 19 chars) ---
str01: MALSYCPKGT
str02: MQSSLNAIPV
str03: MPLSYQHFRK
str04: MEEHVNELHD
str05: MSNFDAIRAL
str06: MFRNQNSRNG
str07: MFYAHAFGGY
str08: MSKFTRRPYQ
str09: MSFVAGVTAQ
str10: MESLVPGFNE

--- Solution (of length 44) ---
  Sol: MEQPSLSKFTRYAEHVNQELNSYCHFDAIRPGFKGVTANEGYQL
str01: M-----------A------L-SYC------P--KG-T-------
str02: M-Q-S-S------------LN------AI-P----V--------
str03: M--P-LS----Y-----Q------HF---R---K----------
str04: ME-----------EHVN-EL----H-D-----------------
str05: M---S-----------N--------FDAIR-------A-----L
str06: M-------F-R-----NQ--NS-------R--------N-G---
str07: M-------F--YA-H------------A----F-G-----GY--
str08: M---S--KFTR------------------RP----------YQ-
str09: M---S---F------V-----------A---G---VTA----Q-
str10: ME--SL---------V--------------PGF-----NE----

solution is feasible: True


In [ ]:
instance_09 = util.parse("protein_n050k050.txt")
solution_09 = solve(instance_09, log=True)


Model:  expressions = 2504, decisions = 2500, constraints = 0, objectives = 1
Param:  time limit = 60 sec, no iteration limit

[objective direction ]:     minimize

[  0 sec,       0 itr]:          536
[ optimality gap     ]:      100.00%


[  1 sec,      20 itr]:          524


[  2 sec,      46 itr]:          509


[  3 sec,      84 itr]:          508


[  4 sec,     116 itr]:          507


...

[ 56 sec,    1954 itr]:          498


[ 57 sec,    1978 itr]:          498


[ 58 sec,    2022 itr]:          498


[ 59 sec,    2058 itr]:          498


[ 60 sec,    2096 itr]:          498
[ optimality gap     ]:      100.00%
[ 60 sec,    2096 itr]:          498
[ optimality gap     ]:      100.00%

2096 iterations performed in 60 seconds

Feasible solution: 
  obj    =          498
  gap    =      100.00%
  bounds =            0


In [ ]:
_instance = instance_09
_solution = solution_09

util.show(_instance)
if _solution is not None:
    util.show(_instance, _solution)
    print(f"solution is feasible: {util.is_feasible(_instance, _solution)}")
else:
    print("--- Solution not found ---")

--- Condition (with 20 chars) ---
str01: MRHLNIDIETYSSNDIKNGVYKYADAEDFEILLFAYSIDGGEVECLDLTR
str02: MERRAHRTHQNWDATKPRERRKQTQHRLTHPDDSIYPRIEKAEGRKEDHG
str03: MEPGAFSTALFDALCDDILHRRLESQLRFGGVQIPPEVSDPRVYAGYALL
str04: MGKFYYSNRRLAVFAQAQSRHLGGSYEQWLACVSGDSAFRAEVKARVQKD
str05: FFRENLAFQQGKAREFPSEEARANSPTSRELWVRRGGNPLSEAGAERRGT
str06: MDPSLTQVWAVEGSVLSAAVDTAETNDTEPDEGLSAENEGETRIIRITGS
str07: MAFDFSVTGNTKLDTSGFTQGVSSMTVAAGTLIADLVKTASSQLTNLAQS
str08: MAVILPSTYTDGTAACTNGSPDVVGTGTMWVNTILPGDFFWTPSGESVRV
str09: MNTGIIDLFDNHVDSIPTILPHQLATLDYLVRTIIDENRSVLLFHIMGSG
str10: MFVFLVLLPLVSSQCVNLRTRTQLPPAYTNSFTRGVYYPDKVFRSSVLHS
str11: MDSKETILIEIIPKIKSYLLDTNISPKSYNDFISRNKNIFVINLYNVSTI
str12: MLLSGKKKMLLDNYETAAARGRGGDERRRGWAFDRPAIVTKRDKSDRMAH
str13: MNGEEDDNEQAAAEQQTKKAKREKPKQARKVTSEAWEHFDATDDGAECKH
str14: MESLVPGFNEKTHVQLSLPVLQVRDVLVRGFGDSVEEVLSEARQHLKDGT
str15: MRYIVSPQLVLQVGKGQEVERALYLTPYDYIDEKSPIYYFLRSHLNIQRP
str16: MPRVPVYDSPQVSPNTVPQARLATPSFATPTFRGADAPAFQDTANQQARQ
str17: MFVFLVLLPLVSSQCVNLRTRTQLPLAYTNS

----------L------------------------IE-----------I----------------------------------I-P--K----I-------K---S-------YL----------L--------D--------------TN---------I---------------S-----------------------------P------K-------S--------Y-------N-D----------------F-IS---------------------------R---------------N-K-----------N--------I----------------F-VI---------NLY--------NV-S----TI--------------
str12: M----L---L----S-------G-K----K-----------------------------------K-----M-L---------L-D---------N------------Y--------------E----------------T----A--------------A---------------A-RG---RG-------------G-------D-----E--------------R----------R--------------------------R--G--------------WA-------------F-------DR----PA----I--VT------------------------K----------------R-----------D---K-------S---D-R-----M--------A---H------------------------------------------------------------------------------------
str13: M------------------N--G---E---------------------------E--D------------------D-------N-------EQ

-----------------E-----------------E-------K---L----KPQ------------L-S---------------------M---------------G-------E------D---V-------QS------------------------------------------------------------------------------------------------------------------------------------------
str26: M-FVFL--VLLPLVSSQCVN--------L-----------ITR------T------------Q--------------S---------YT------NSFTRG-V-----Y------------------Y--------------------PDK--V----------F----R----S------------S-----V-------LH--------S-T-Q--------D---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
str27: M-----------------------K--------------------F-----------DVLS--L------F---A---P---------------W----------A-------K-V-------------------D------EQ--------E---------Y-----D-----------Q-------------------------------

-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
str40: M-----ES-L---V-------PGF-NE--KTHVQLSLP-----VL-QV---------------------------------------------------------------C-----------------------D----------V--------L---------------------VRG-----------------------------F-------G------D-----------S--------V-------------E----------E-------------------VL-S--------E----------A---R--------------Q-H---L--------K-------DG----------T----------------------------------------------------------------------------------------------------------------------------------
str41: M------------------N-----N-------Q----R--------------------------K------------------------K-------T------A------------R--PS--FN-----------M-----L-----K------R--A--------R---N----R------V-S-------T------------V--S---QL------A--KR----------------------FS---------------KG--